In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from linearmodels.panel import RandomEffects
from linearmodels.panel import PanelOLS

In [25]:
df = pd.read_csv("..\Sales_no_outliers_brewery_only.csv")
print(df.shape)
df.head()

(91, 37)


,Unnamed: 0,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Revenue,Operating Income,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
0,516,2019,BUDWEISER BREWING CO,CAYMAN ISLANDS,Brewers,Manufacture of beverages,-7.019612,-37.638941,6.582921e+09,1.227706e+09,...,-2.484940e+08,-21847.771988,-140.450129,-28.052546,-25278.227923,-167.406661,1886.414782,1886.414782,-8773.429720,0.017490
1,517,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-6.596233,-46.522794,1.057940e+10,1.500000e+09,...,-3.977745e+08,-77571.792798,-1113.208360,-326.988191,-37024.568588,-1326.865949,15176.840942,15176.840942,-12261.768244,0.059202
2,519,2019,COMPANIA CERVECERIAS UNIDAS,CHILE,Brewers,Manufacture of beverages,-6.415653,-54.146661,2.423979e+09,2.872090e+08,...,-9.485571e+07,-65916.007408,-104.142317,-467.109898,-5751.475432,-124.130306,-1430.679992,-55213.362037,-2224.738635,0.084882
3,523,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-3.388736,-11.492977,5.248464e+10,1.547524e+10,...,-1.909141e+08,-222519.248882,-3914.291085,-325.789140,-190769.424529,-4665.559245,56659.806086,56659.806086,-64862.799133,0.080199
4,526,2019,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-2.414563,-16.054846,9.890572e+09,1.487489e+09,...,-3.902679e+07,-366663.271634,-1480.449400,-288.844325,-27951.868900,-1764.591400,19253.589023,19132.282417,-8148.334758,0.029599


## Helper Functions

In [37]:
harvard_set = ["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]

sdg_set = ["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]

sdg_finalset = ["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 3.3", "SDG 3.4", "SDG 6", "SDG 15.2"]

depVar_EISales = "Environmental Intensity (Sales)"
depVar_EIOpInc = "Environmental Intensity (Op Inc)"
depVar_Revenue = "Revenue"
depVar_OpInc = "Operating Income"


def getXYVal(selectedSet, dependent_variable, df_source):
    """
        Sets up the dependent and independent variables to be used by the regressions

    Args:
        isHarvard (bool): check if using the harvard targets or SDG targets
        isRevenue (bool): check if using the revenue or operating income
        isAbsolute (bool, optional): Optional to check if using the absolute values or percentage. Defaults to False.

    Returns:
        None
    """    
    df_source = df_source 
        
    X = df_source[selectedSet]
    X = sm.add_constant(X)
    
    y = df_source[dependent_variable]
    
    return X, y

In [31]:
def runPanelOLS(selectedSet, dependent_variable, df_source):
    """
        Runs the lasso regression with the settings passed as parameters.

    Args:
        isHarvard (bool): check if using the harvard targets or SDG targets
        isRevenue (bool): check if using the revenue or operating income
        isAbsolute (bool, optional): Optional to check if using the absolute values or percentage. Defaults to False.

    Returns:
        None
    """
    df_source.set_index(['Company Name', 'Year'], inplace=True)
    
    X,y = getXYVal(selectedSet=selectedSet, dependent_variable=dependent_variable, df_source=df_source)
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
    model = PanelOLS(y, X, check_rank=False)

    # Fit the model
    results = model.fit()
    
    # Print summary statistics
    print(results)

## Panel OLS
### All SDGs

In [33]:
runPanelOLS(selectedSet=sdg_set, dependent_variable=depVar_EISales, df_source=df.copy())

                                 PanelOLS Estimation Summary                                 
Dep. Variable:     Environmental Intensity (Sales)   R-squared:                        0.9047
Estimator:                                PanelOLS   R-squared (Between):              0.9766
No. Observations:                               91   R-squared (Within):               0.3370
Date:                             Sat, Oct 07 2023   R-squared (Overall):              0.9047
Time:                                     11:46:59   Log-likelihood                   -65.114
Cov. Estimator:                         Unadjusted                                           
                                                     F-statistic:                      40.783
Entities:                                       12   P-value                           0.0000
Avg Obs:                                    7.5833   Distribution:                   F(17,73)
Min Obs:                                    1.0000          

In [38]:
runPanelOLS(selectedSet=sdg_finalset, dependent_variable=depVar_EISales, df_source=df.copy())

                                 PanelOLS Estimation Summary                                 
Dep. Variable:     Environmental Intensity (Sales)   R-squared:                        0.7983
Estimator:                                PanelOLS   R-squared (Between):              0.9199
No. Observations:                               91   R-squared (Within):              -0.1198
Date:                             Sat, Oct 07 2023   R-squared (Overall):              0.7983
Time:                                     11:56:53   Log-likelihood                   -99.247
Cov. Estimator:                         Unadjusted                                           
                                                     F-statistic:                      46.928
Entities:                                       12   P-value                           0.0000
Avg Obs:                                    7.5833   Distribution:                    F(7,83)
Min Obs:                                    1.0000          

In [14]:
runPanelOLS(selectedSet="SDG 6", dependent_variable=depVar_EISales, df_source=df.copy())

                                 PanelOLS Estimation Summary                                 
Dep. Variable:     Environmental Intensity (Sales)   R-squared:                        0.4900
Estimator:                                PanelOLS   R-squared (Between):              0.5828
No. Observations:                               91   R-squared (Within):              -0.2037
Date:                             Sat, Oct 07 2023   R-squared (Overall):              0.4900
Time:                                     11:25:26   Log-likelihood                   -141.45
Cov. Estimator:                         Unadjusted                                           
                                                     F-statistic:                      85.514
Entities:                                       12   P-value                           0.0000
Avg Obs:                                    7.5833   Distribution:                    F(1,89)
Min Obs:                                    1.0000          